In [1]:
import urllib 


In [24]:
sample_list = "GSE15745,GSE17448,GSE19711,GSE19711,GSE20067,GSE20236,GSE20242,GSE20242,GSE22595,GSE25892,GSE26033,GSE26033   ,GSE26126,GSE26974   ,GSE27097,GSE27317,GSE28746,GSE30090,GSE30456,GSE30601,GSE30653,GSE30758,GSE30870,GSE31848,GSE32146,GSE32149,GSE32393,GSE34035,GSE34035,GSE34257,GSE34639,GSE35069,GSE36064,GSE36166,GSE36642,GSE36642,GSE36642,GSE36812,GSE37008,GSE37066,GSE37988,GSE38291,GSE38291,GSE38608,GSE38608,GSE38873,GSE40279,GSE40700,GSE41037,GSE41169,GSE41782,GSE41826,GSE42510,GSE42700,GSE42700,GSE42861,GSE42865,\
GSE43269,GSE44667,GSE47627"
sample_arr = sample_list.split(",")
len(sample_arr)

60

In [25]:
#ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE40nnn/GSE40700/matrix/GSE40700_series_matrix.txt.gz
for i in sample_arr:
    key_num = i.split("GSE")[1]
    url_key= "GSE"+key_num[0:2]
    file_n = i + "_series_matrix.txt.gz"
    ftp= "ftp://ftp.ncbi.nlm.nih.gov/geo/series/"+url_key.replace(" ","")+ "/matrix/" + file_n
